# Парсер для сайта zakupki.gov.ru

[zakupki.gov.ru](https://zakupki.gov.ru/epz/order/extendedsearch/results.html?morphology=on&search-filter=Дате+размещения&pageNumber=1&sortDirection=false&recordsPerPage=_50&showLotsInfoHidden=false&sortBy=UPDATE_DATE&fz44=on&fz223=on&af=on&ca=on&pc=on&pa=on&currencyIdGeneral=-1&publishDateFrom=01.01.2024&publishDateTo=31.01.2024&okpd2IdsWithNested=on&okpd2Ids=8873937%2C8873938&okpd2IdsCodes=62%2C63)

In [1]:
import pandas as pd
import time

from selenium import webdriver
# from selenium.webdriver import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By

from tqdm import tqdm

In [3]:
%%time


# url с параметрами поиска данных, включающих:
# - 50 строк вывода данных на странице
# - дату размещения с 01.01.2024 по 31.01.2024
# - ОКПД2 = 62 и 63 с учетом вложенных
url = """
https://zakupki.gov.ru/epz/order/extendedsearch/results.html?morphology=on&
search-filter=Дате+размещения&pageNumber=1&sortDirection=false&recordsPerPage=_50&
showLotsInfoHidden=false&sortBy=UPDATE_DATE&fz44=on&fz223=on&af=on&ca=on&pc=on&
pa=on&currencyIdGeneral=-1&publishDateFrom=01.01.2024&publishDateTo=31.01.2024&
okpd2IdsWithNested=on&okpd2Ids=8873937%2C8873938&okpd2IdsCodes=62%2C63
"""

with webdriver.Chrome() as browser:
    browser.maximize_window()
    browser.get(url)
    time.sleep(2)
    data = []

    while True:
        el_block = browser.find_element(By.CLASS_NAME, 'search-registry-entrys-block')
        all_zak = el_block.find_elements(By.CLASS_NAME, 'search-registry-entry-block')

        for zak in tqdm(all_zak):
            element = {}
            num = zak.find_element(By.CLASS_NAME, 'registry-entry__header-mid__number')
            num = num.find_element(By.TAG_NAME, 'a')
            order_url = num.get_attribute('href')
            obj_zak = zak.find_element(By.CLASS_NAME, 'registry-entry__body-value').text
            price = zak.find_element(By.CLASS_NAME, 'price-block__value').text
            customer = zak.find_element(By.CLASS_NAME, 'registry-entry__body-href')
            customer_name = customer.text
            customer_url = customer.find_element(By.TAG_NAME, 'a').get_attribute('href')
            element['order_id'] = num.text.strip().replace('№', '')
            element['obj_zak'] = obj_zak.strip()
            element['price'] = price.strip().replace('₽', '').replace('$', '').replace(',', '.').replace(' ', '')
            element['customer'] = customer_name.strip()
            element['order_url'] = order_url
            element['customer_url'] = customer_url

            data.append(element)

        next_page = browser.find_elements(By.CSS_SELECTOR, "a.paginator-button.paginator-button-next")
        if len(next_page) == 0:
            break

        browser.execute_script("arguments[0].click();", next_page[0])
        time.sleep(5)

print(len(data))

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:02<00:00, 17.15it/s]


1000
CPU times: total: 8.91 s
Wall time: 2min 40s


In [9]:
data

[{'order_id': ' 0338200006524000003',
  'obj_zak': 'Оказание услуг по информационному обслуживанию (обновлению) ранее используемой справочно-правовой системы «Гарант»',
  'price': '103444.00',
  'customer': 'КРАЕВОЕ ГОСУДАРСТВЕННОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ПО ПЛЕМЕННОЙ РАБОТЕ "КАМЧАТСКОЕ"',
  'order_url': 'https://zakupki.gov.ru/epz/order/notice/ea20/view/common-info.html?regNumber=0338200006524000003',
  'customer_url': 'https://zakupki.gov.ru/epz/organization/view/info.html?organizationId=736588'},
 {'order_id': ' 0361200015024000158',
  'obj_zak': 'Оказание услуг по сопровождению программного обеспечения (программ для ЭВМ) информационной подсистемы мониторинга родовспоможения Регионального фрагмента Единой государственной информационной системы в сфере здравоохранения Сахалинской области',
  'price': '4600000.02',
  'customer': 'ГОСУДАРСТВЕННОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ "ЦЕНТР ГОСУДАРСТВЕННЫХ ЗАКУПОК САХАЛИНСКОЙ ОБЛАСТИ"',
  'order_url': 'https://zakupki.gov.ru/epz/order/notice/ea20/view/commo

In [10]:
df_pars = pd.DataFrame(data)
df_pars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   order_id      1000 non-null   object
 1   obj_zak       1000 non-null   object
 2   price         1000 non-null   object
 3   customer      1000 non-null   object
 4   order_url     1000 non-null   object
 5   customer_url  1000 non-null   object
dtypes: object(6)
memory usage: 47.0+ KB


In [11]:
df_pars

,order_id,obj_zak,price,customer,order_url,customer_url
0,0338200006524000003,Оказание услуг по информационному обслуживанию...,103444.00,КРАЕВОЕ ГОСУДАРСТВЕННОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ПО...,https://zakupki.gov.ru/epz/order/notice/ea20/v...,https://zakupki.gov.ru/epz/organization/view/i...
1,0361200015024000158,Оказание услуг по сопровождению программного о...,4600000.02,"ГОСУДАРСТВЕННОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ""ЦЕНТР ГОС...",https://zakupki.gov.ru/epz/order/notice/ea20/v...,https://zakupki.gov.ru/epz/organization/view/i...
2,0191300011024000001,Работы по актуализации на 2025 год Схемы тепло...,4988333.00,УПРАВЛЕНИЕ РЕГУЛИРОВАНИЯ ЦЕН И ТАРИФОВ ГОРОДСК...,https://zakupki.gov.ru/epz/order/notice/ok20/v...,https://zakupki.gov.ru/epz/organization/view/i...
3,0302300048724000052,Оказание информационных услуг по освещению дея...,602000.00,"МУНИЦИПАЛЬНОЕ УЧРЕЖДЕНИЕ ""УПРАВЛЕНИЕ ПО МУНИЦИ...",https://zakupki.gov.ru/epz/order/notice/zk20/v...,https://zakupki.gov.ru/epz/organization/view/i...
4,0351100008224000007,ОКАЗАНИЕ УСЛУГ ПО АДАПТАЦИИ И СОПРОВОЖДЕНИЮ ЭК...,192882.69,"ФЕДЕРАЛЬНОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ""ИСПРАВИТЕЛЬНА...",https://zakupki.gov.ru/epz/order/notice/ea20/v...,https://zakupki.gov.ru/epz/organization/view/i...
...,...,...,...,...,...,...
995,0372200211124000006,Оказание услуг по сопровождению и технической ...,1100732.00,САНКТ-ПЕТЕРБУРГСКОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ...,https://zakupki.gov.ru/epz/order/notice/ea20/v...,https://zakupki.gov.ru/epz/organization/view/i...
996,0372200211124000005,Оказание услуг по настройке функциональных мех...,309750.00,САНКТ-ПЕТЕРБУРГСКОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ...,https://zakupki.gov.ru/epz/order/notice/zk20/v...,https://zakupki.gov.ru/epz/organization/view/i...
997,0372200211124000004,Оказание услуг по настройке функциональных мех...,168000.00,САНКТ-ПЕТЕРБУРГСКОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ...,https://zakupki.gov.ru/epz/order/notice/zk20/v...,https://zakupki.gov.ru/epz/organization/view/i...
998,32413230682,оказание услуг по установке программного обесп...,8888956.04,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕН...,https://zakupki.gov.ru/epz/order/notice/notice...,https://zakupki.gov.ru/epz/organization/view22...


In [12]:
df_pars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   order_id      1000 non-null   object
 1   obj_zak       1000 non-null   object
 2   price         1000 non-null   object
 3   customer      1000 non-null   object
 4   order_url     1000 non-null   object
 5   customer_url  1000 non-null   object
dtypes: object(6)
memory usage: 47.0+ KB


In [13]:
# сохраним спарсенный датасет
try:
    df_pars.to_csv('C:/Users/lorad/OneDrive/Documents/Моя папка/Data Science/Курсы/'
              '1Т Data. DS (Архитектор ИИ)/_Дата-стажировка 2.0/Задание/zakupki_50.csv', index=False, encoding='utf-8-sig')
except:
    df_pars.to_csv('D:/Юлия/Data Science/Курсы/'
          '1Т Data. DS (Архитектор ИИ)/_Дата-стажировка 2.0/Задание/zakupki_50.csv', index=False, encoding='utf-8-sig')

In [14]:
# загрузим датасет
try:
    df = pd.read_csv(r'C:/Users/lorad/OneDrive/Documents/Моя папка/Data Science/Курсы/'
                     '1Т Data. DS (Архитектор ИИ)/_Дата-стажировка 2.0/Задание/zakupki_50.csv')
except:
    try:
        df = pd.read_csv(r'D:/Юлия/Data Science/Курсы/'
                         '1Т Data. DS (Архитектор ИИ)/_Дата-стажировка 2.0/Задание/zakupki_50.csv')
    except:
        df = pd.read_csv('...')

In [15]:
df

,order_id,obj_zak,price,customer,order_url,customer_url
0,338200006524000003,Оказание услуг по информационному обслуживанию...,103444.00,КРАЕВОЕ ГОСУДАРСТВЕННОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ПО...,https://zakupki.gov.ru/epz/order/notice/ea20/v...,https://zakupki.gov.ru/epz/organization/view/i...
1,361200015024000158,Оказание услуг по сопровождению программного о...,4600000.02,"ГОСУДАРСТВЕННОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ""ЦЕНТР ГОС...",https://zakupki.gov.ru/epz/order/notice/ea20/v...,https://zakupki.gov.ru/epz/organization/view/i...
2,191300011024000001,Работы по актуализации на 2025 год Схемы тепло...,4988333.00,УПРАВЛЕНИЕ РЕГУЛИРОВАНИЯ ЦЕН И ТАРИФОВ ГОРОДСК...,https://zakupki.gov.ru/epz/order/notice/ok20/v...,https://zakupki.gov.ru/epz/organization/view/i...
3,302300048724000052,Оказание информационных услуг по освещению дея...,602000.00,"МУНИЦИПАЛЬНОЕ УЧРЕЖДЕНИЕ ""УПРАВЛЕНИЕ ПО МУНИЦИ...",https://zakupki.gov.ru/epz/order/notice/zk20/v...,https://zakupki.gov.ru/epz/organization/view/i...
4,351100008224000007,ОКАЗАНИЕ УСЛУГ ПО АДАПТАЦИИ И СОПРОВОЖДЕНИЮ ЭК...,192882.69,"ФЕДЕРАЛЬНОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ""ИСПРАВИТЕЛЬНА...",https://zakupki.gov.ru/epz/order/notice/ea20/v...,https://zakupki.gov.ru/epz/organization/view/i...
...,...,...,...,...,...,...
995,372200211124000006,Оказание услуг по сопровождению и технической ...,1100732.00,САНКТ-ПЕТЕРБУРГСКОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ...,https://zakupki.gov.ru/epz/order/notice/ea20/v...,https://zakupki.gov.ru/epz/organization/view/i...
996,372200211124000005,Оказание услуг по настройке функциональных мех...,309750.00,САНКТ-ПЕТЕРБУРГСКОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ...,https://zakupki.gov.ru/epz/order/notice/zk20/v...,https://zakupki.gov.ru/epz/organization/view/i...
997,372200211124000004,Оказание услуг по настройке функциональных мех...,168000.00,САНКТ-ПЕТЕРБУРГСКОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ...,https://zakupki.gov.ru/epz/order/notice/zk20/v...,https://zakupki.gov.ru/epz/organization/view/i...
998,32413230682,оказание услуг по установке программного обесп...,8888956.04,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕН...,https://zakupki.gov.ru/epz/order/notice/notice...,https://zakupki.gov.ru/epz/organization/view22...


In [16]:
df['order_id'].nunique()

978

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   order_id      1000 non-null   int64  
 1   obj_zak       1000 non-null   object 
 2   price         1000 non-null   float64
 3   customer      1000 non-null   object 
 4   order_url     1000 non-null   object 
 5   customer_url  1000 non-null   object 
dtypes: float64(1), int64(1), object(4)
memory usage: 47.0+ KB
